<div class="alert alert-warning">
This work consists of two parts:     
    <ul>
        <li> It is a modified version of the original fastai API mixup code for the introduction of meta data in training. </li>
         <li> If you find it helpful, please give a upvote! It will inspire me to share more instereting work! </li>
        <li> Can be applied in the notebook: https://www.kaggle.com/calvchen/swin-transformer-224-meta with cbs=[MixUp(0.2)].</a></li>
    </ul>
    
</div>


    






In [ ]:
from fastai.vision.all import *

class MixUp(MixHandler):
    "Implementation of https://arxiv.org/abs/1710.09412"
    def __init__(self, alpha=.4): super().__init__(alpha)
    def before_batch(self):
        lam = self.distrib.sample((self.y.size(0),)).squeeze().to(self.x[0].device)
        lam = torch.stack([lam, 1-lam], 1)
        self.lam = lam.max(1)[0]
        shuffle = torch.randperm(self.y.size(0)).to(self.x[0].device)
        xb1, xb2, self.yb1 = list(L(self.xb[0]).itemgot(shuffle)), list(L(self.xb[1]).itemgot(shuffle)), list(L(self.yb).itemgot(shuffle))

        nx_dims = len(self.x[0].size())
#         print(nx_dims) # 4

        a = L(xb1, self.xb[0])
        b = L(xb2, self.xb[1])
        
        a = a.map_zip(torch.lerp, weight=unsqueeze(self.lam, n=nx_dims-1))
        b = b.map_zip(torch.lerp, weight=unsqueeze(self.lam, n=nx_dims-3))   

        self.learn.xb = (a[0], b[0])

        
        if not self.stack_y:
            ny_dims = len(self.y.size())
            self.learn.yb = list(L(self.yb1, self.yb).map_zip(torch.lerp,weight=unsqueeze(self.lam, n=ny_dims-1)))
